#### (1) How many records are in the file? 

A record in a FASTA file is defined as a single-line header, followed by lines of sequence data. The header line is distinguished from the sequence data by a greater-than (">") symbol in the first column. The word following the ">" symbol is the identifier of the sequence, and the rest of the line is an optional description of the entry.

In [1]:
try:
    with open("dna.example.fasta") as f:
        data = f.read()
        data = data.split('\n')
except IOError:
    print("File does not exist!")


In [2]:
seqs = {}
for line in data[:-1]:
    line = line.rstrip()
    # distinguish header from sequence
    if line[0] == '>':    # or line.startswith('>')
        words = line.split()
        name = words[0][1:]  # We don't want to have ">" sign
        seqs[name] = ""
    else:
        seqs[name] = seqs[name] + line
    

In [3]:
len(seqs)

25

#### (2) What are the lengths of the sequences in the file?

In [4]:
seqsvalue = list(seqs.values())
seqskey = list(seqs.keys())
for n, i in enumerate(seqsvalue):
    print seqskey[n], len(i)


gi|142022655|gb|EQ086233.1|59 4745
gi|142022655|gb|EQ086233.1|422 1801
gi|142022655|gb|EQ086233.1|245 1325
gi|142022655|gb|EQ086233.1|507 2124
gi|142022655|gb|EQ086233.1|221 2863
gi|142022655|gb|EQ086233.1|564 1663
gi|142022655|gb|EQ086233.1|378 555
gi|142022655|gb|EQ086233.1|229 3072
gi|142022655|gb|EQ086233.1|294 3832
gi|142022655|gb|EQ086233.1|160 724
gi|142022655|gb|EQ086233.1|279 1189
gi|142022655|gb|EQ086233.1|323 4805
gi|142022655|gb|EQ086233.1|43 990
gi|142022655|gb|EQ086233.1|41 3080
gi|142022655|gb|EQ086233.1|101 2449
gi|142022655|gb|EQ086233.1|384 3603
gi|142022655|gb|EQ086233.1|438 3424
gi|142022655|gb|EQ086233.1|455 691
gi|142022655|gb|EQ086233.1|210 1451
gi|142022655|gb|EQ086233.1|237 3276
gi|142022655|gb|EQ086233.1|158 1608
gi|142022655|gb|EQ086233.1|280 2478
gi|142022655|gb|EQ086233.1|319 1810
gi|142022655|gb|EQ086233.1|521 512
gi|142022655|gb|EQ086233.1|350 1712


####(3) What is the longest sequence and what is the shortest sequence?

In [5]:
lengthlist = [len(i) for i in seqsvalue]
print max(lengthlist)
print min(lengthlist)


4805
512


####(4) Is there more than one longest or shortest sequence? What are their identifiers? 

In [6]:
print lengthlist.count(4805)
print lengthlist.count(512)


1
1


In [7]:
for n, i in enumerate(seqsvalue):
    if len(i) == 4805:
        print seqskey[n]
    elif len(i) == 512:
        print seqskey[n]
    else:
        continue


gi|142022655|gb|EQ086233.1|323
gi|142022655|gb|EQ086233.1|521


####(5) What is the length of the longest open reading frame(ORF) in the file? 
In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons). Depending on where we start, there are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5'). For instance, the three possible forward reading frames for the sequence AGGTGACACCGCAAGCCTTATATTAGC are: 


AGG TGA CAC CGC AAG CCT TAT ATT AGC

A GGT GAC ACC GCA AGC CTT ATA TTA GC

AG GTG ACA CCG CAA GCC TTA TAT TAG C 


These are called reading frames 1, 2, and 3 respectively

An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). For instance, ATGAAATAG is an ORF of length 9.

In [8]:
def is_ORF(dna, frame):
    start_codon_found = False
    stop_codon_found = False
    start_codon = ['ATG']
    stop_codon = ['TAA', 'TAG', 'TGA']

    # Assume only consider forward strand
    start = []
    end = []
    for i in range(frame-1, len(dna), 3):
        codon = dna[i:i+3].upper()
        if codon in start_codon:
            start_codon_found = True
            start.append(i)
        elif codon in stop_codon:
            stop_codon_found = True
            end.append(i)
        else:
            pass

    # Find all possible start and end position and length of ORF
    total = start+end
    newline = sorted(total)
    possi_position = []

    for n, i in enumerate(newline):
        for x in newline[n:]:
            if i in start and x in end:
                possi_position.append([i, x, (abs(i-x)+3)])
                break

    # Assign 0 to empty list. No start and stop codon.
    if possi_position == []:
        possi_position.append([0, 0, 0])

    return (start_codon_found and stop_codon_found), possi_position


In [9]:
print is_ORF('ATGAAATAG', 1)

(True, [[0, 6, 9]])


In [10]:
def max_min_len(lst):
    """ This function will return [start, stop, maxlength],
        [start, stop, minlength], maxlength, minlength
    """
    alllength = []
    for i in lst:
        alllength.append(i[2])
    max_ = max(alllength)
    min_ = min(alllength)

    allmax = []
    allmin = []
    for n, i in enumerate(lst):
        if i[2] == max_:
            allmax.append(lst[n])
        if i[2] == min_:
            allmin.append(lst[n])

    return allmax, allmin, max_, min_


In [11]:
# Longest ORF in each sequences

n = -1
for dna in seqsvalue:
    n += 1
    longest = []
    for i in range(1, 4):
        lst = is_ORF(dna, i)[1]
        longest.append(max_min_len(lst)[2])
    print ('For sequence, %s' % ((seqskey[n])))
    print ('The longest ORF has %s, nt.' % (max(longest)))


For sequence, gi|142022655|gb|EQ086233.1|59
The longest ORF has 1107, nt.
For sequence, gi|142022655|gb|EQ086233.1|422
The longest ORF has 639, nt.
For sequence, gi|142022655|gb|EQ086233.1|245
The longest ORF has 636, nt.
For sequence, gi|142022655|gb|EQ086233.1|507
The longest ORF has 741, nt.
For sequence, gi|142022655|gb|EQ086233.1|221
The longest ORF has 594, nt.
For sequence, gi|142022655|gb|EQ086233.1|564
The longest ORF has 507, nt.
For sequence, gi|142022655|gb|EQ086233.1|378
The longest ORF has 534, nt.
For sequence, gi|142022655|gb|EQ086233.1|229
The longest ORF has 1311, nt.
For sequence, gi|142022655|gb|EQ086233.1|294
The longest ORF has 1608, nt.
For sequence, gi|142022655|gb|EQ086233.1|160
The longest ORF has 363, nt.
For sequence, gi|142022655|gb|EQ086233.1|279
The longest ORF has 153, nt.
For sequence, gi|142022655|gb|EQ086233.1|323
The longest ORF has 1686, nt.
For sequence, gi|142022655|gb|EQ086233.1|43
The longest ORF has 213, nt.
For sequence, gi|142022655|gb|EQ0862

In [12]:
# Find longest ORF length of all sequences in festa file
find_longest = []
for dna in seqsvalue:
    for i in range(1, 4):
        lst = is_ORF(dna, i)[1]
        find_longest.append(max_min_len(lst)[2])
print max(find_longest)


1686


####(6) For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier?                            
####(7) What is the starting position of the longest ORF in the sequence that contains it? 

In [13]:
key = 'gi|142022655|gb|EQ086233.1|521'
dna = seqs[key]

find_longest = []
for i in range(1, 4):
    lst = is_ORF(dna, i)[1]
    find_longest.append([max_min_len(lst)[2], max_min_len(lst)[0]])

print max(find_longest)


[159, [[125, 281, 159]]]


start at codon (125/3)+1 =42 ; end at codon (281/3)+1 = 94

#### (8) Given a length n, identify all repeats of length n in all sequences in the FASTA file 
#### (9) Determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length
A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence. Although repeats can occur on both the forward and reverse strands of the DNA sequence, we will only consider repeats on the forward strand here. Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and once at position 3. 

In [14]:
# Find possible seqs in that given length n
def find_motif(seq, n):
    motif = []
    for i in range(0, len(seq)-n+1):
        value = seq[i:i+n]
        motif.append(value)

    sett = set(motif)

    return sett


In [21]:
# Assume n=6
total = {}
for dna in seqsvalue:
    motif = find_motif(dna, 6)
    for i in motif:
        if i in dna:
            if i not in total.keys():
                total[i] = 0
                total[i] = total[i]+dna.count(i)
            else:
                total[i] = total[i]+dna.count(i)
# Remove seq only shows one time
for name, value in total.items():
    if value == 1:
        del total[name]

max_ = max(total.values())
ttee = total.values()

print total, max_


{'GAACGT': 12, 'CTTCTT': 7, 'CACCCT': 8, 'GAACGG': 47, 'GAACGC': 40, 'GAACGA': 21, 'CACCCA': 11, 'CTTCTA': 4, 'CTTCTC': 5, 'CACCCG': 19, 'CTTCTG': 8, 'CGTGTG': 12, 'CGTGTC': 18, 'TGAGTG': 4, 'CGTGTA': 9, 'GTTGTG': 2, 'AGGCAA': 8, 'GGAAAT': 12, 'CGTGTT': 14, 'TCACTG': 7, 'GTCAAA': 2, 'GTCAAC': 12, 'GTCAAG': 10, 'CTGTCC': 12, 'TCAGAG': 2, 'CTGTCA': 7, 'CTGTCG': 24, 'TCAGAC': 9, 'TCAGAA': 3, 'GTCAAT': 6, 'TCAGAT': 9, 'GTATCG': 20, 'CTGTCT': 4, 'GGTGTC': 14, 'GGTGTA': 3, 'GGTGTG': 11, 'CCGGGG': 19, 'TTCTGT': 6, 'GTATTG': 5, 'CCGGGC': 45, 'CCGGGA': 11, 'TATCCA': 2, 'TATCCC': 4, 'GTTCTG': 12, 'TATCCG': 11, 'GGTGTT': 17, 'ATTCCG': 17, 'TTCTGG': 16, 'CCGGGT': 30, 'ATTCCC': 10, 'TTCTGC': 8, 'ATTCCA': 8, 'TTCTGA': 6, 'TGCACT': 5, 'TATCTT': 6, 'CGCGGT': 47, 'CGCGGG': 43, 'CGCGGC': 134, 'CGCGGA': 46, 'ACCTGT': 8, 'TCATGT': 11, 'CCCGCC': 28, 'CTCCGC': 3, 'TTACAC': 3, 'CTCCGG': 12, 'TCATGC': 9, 'ACCTGG': 11, 'TCATGA': 16, 'TCATGG': 12, 'ACCTGC': 29, 'ACCTGA': 5, 'GTATTT': 8, 'CTCGAT': 39, 'CTCCGT': 

In [22]:
for name, value in total.items():
    if value == max_:
        print name


CGGCGC
